In [120]:
import pandas as pd
import numpy as np

In [121]:
data = pd.read_csv("car.csv",names=["buying", "maint", "doors", "persons", "lug_boot", "safety", "decision"])

In [122]:
data.head()

,buying,maint,doors,persons,lug_boot,safety,decision
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [123]:
data["buying"].replace(["vhigh", "high", "med", "low"], [4, 3, 2, 1], inplace=True)
data["maint"].replace(["vhigh", "high", "med", "low"], [4, 3, 2, 1], inplace=True)
data["doors"].replace(["2", "3", "4", "5more"], [2, 3, 4, 6], inplace=True)
data["persons"].replace(["2", "4", "more"], [2, 4, 5], inplace=True)
data["lug_boot"].replace(["small", "med", "big"], [0, 1, 2], inplace=True)
data["safety"].replace(["low", "med", "high"], [0, 1, 2], inplace=True)
data["decision"].replace(["unacc", "acc", "good", "vgood"], [0, 1, 2, 3], inplace=True)

data['buying'] = data['buying'].astype(int)
data['maint'] = data['maint'].astype(int)
data['doors'] = data['doors'].astype(int)
data['persons'] = data['persons'].astype(int)
data['lug_boot'] = data['lug_boot'].astype(int)
data['safety'] = data['safety'].astype(int)
data['decision'] = data['decision'].astype(int)

In [124]:
from sklearn.model_selection import StratifiedKFold

for i in range(10):

    data["kfold"] = -1

    print(data.head())

    data = data.sample(frac=1).reset_index(drop=True)
    kf = StratifiedKFold(n_splits=5, shuffle=True)

    for folds, (train_idx, test_idx) in enumerate(
           kf.split(X=data, y=data.decision.values)):
    
           print("train_idx : ", len(train_idx), "test_idx :", len(test_idx))
           data.loc[test_idx, "kfold"] = folds

data.to_csv("ID3/ctrain_folds.csv", index=False)


   buying  maint  doors  persons  lug_boot  safety  decision  kfold
0       4      4      2        2         0       0         0     -1
1       4      4      2        2         0       1         0     -1
2       4      4      2        2         0       2         0     -1
3       4      4      2        2         1       0         0     -1
4       4      4      2        2         1       1         0     -1
train_idx :  1382 test_idx : 346
train_idx :  1382 test_idx : 346
train_idx :  1382 test_idx : 346
train_idx :  1382 test_idx : 346
train_idx :  1384 test_idx : 344
   buying  maint  doors  persons  lug_boot  safety  decision  kfold
0       3      1      6        2         0       0         0     -1
1       3      1      2        4         2       1         1     -1
2       2      1      2        2         2       1         0     -1
3       1      4      6        5         1       2         1     -1
4       3      3      2        5         2       1         1     -1
train_idx :  1382 t

In [125]:
new_df = pd.read_csv("ID3/ctrain_folds.csv")

In [126]:
new_df.head()

,buying,maint,doors,persons,lug_boot,safety,decision,kfold
0,4,3,6,5,0,1,0,3
1,2,1,3,4,1,1,1,4
2,3,1,2,2,2,2,0,1
3,4,1,2,5,2,2,1,0
4,1,3,2,2,2,0,0,3


In [127]:
train_df = new_df[(new_df['kfold'] == 0) | (new_df['kfold'] == 1) | (new_df['kfold'] == 2) | (new_df['kfold'] == 3)]
test_df = new_df[(new_df['kfold']==4)]


In [128]:
#read_data.columns[0]
data.keys()[0]

'buying'

# Entrophy of the Training Data Set

In [129]:
#This function is used to calculate the entropy of probaility

def p_entropy(probs):  
    import math
    return sum( [-prob*math.log(prob, 2) for prob in probs] )

#This function is to calulate the entropy of the given Data Sets/List w.r.t target attributes
def entropy_of_target(a_list):  
    
    from collections import Counter
    count = Counter(x for x in a_list)   # Counter is used to calculate the propotion of class
   
    num_instances = len(a_list)*1.0   
    print("\n No of Instances of the Sub Class is {0}:".format(num_instances ))
    probs = [x / num_instances for x in count.values()]  # x means no of YES/NO
    print("\n Classes:",min(count),max(count))
    print(" \n Probabilities of Class {0} is {1}:".format(min(count),min(probs)))
    print(" \n Probabilities of Class {0} is {1}:".format(max(count),max(probs)))
    return p_entropy(probs) # Calling Entropy 
    
# The initial entropy of the YES/NO attribute for our dataset.
print("\n  INPUT DATA SET FOR ENTROPY CALCULATION:\n", train_df['kfold'])

total_entropy = entropy_of_target(train_df['kfold'])

print("\n Total Entropy of mushroom Data Set:",total_entropy)


  INPUT DATA SET FOR ENTROPY CALCULATION:
 0       3
2       1
3       0
4       3
7       1
       ..
1723    2
1724    0
1725    0
1726    1
1727    3
Name: kfold, Length: 1384, dtype: int64

 No of Instances of the Sub Class is 1384.0:

 Classes: 0 3
 
 Probabilities of Class 0 is 0.25:
 
 Probabilities of Class 3 is 0.25:

 Total Entropy of mushroom Data Set: 2.0


# Information Gain of Attributes

In [130]:
def info_gain(dataf, split_attr_name, target_attr_name, trace=0):
    print("Information Gain Calculation of ",split_attr_name)
    
    #Takes a DataFrame of attributes, and quantifies the entropy of a target
    #attribute after performing a split along the values of another attribute.
    
    # Spliting Data w.r.t attribute
    dataf_split = dataf.groupby(split_attr_name)
   
    
    # Calculating Entropy for Target Attribute also
   
    nobs = len(dataf.index) * 1.0
  
    dataf_agg_ent = dataf_split.agg({target_attr_name : [entropy_of_target, lambda x: len(x)/nobs] })[target_attr_name]
    
    dataf_agg_ent.columns = ['Entropy', 'PropObservations']
    
    
    # Calculate Information Gain:
    new_entropy = sum( dataf_agg_ent['Entropy'] * dataf_agg_ent['PropObservations'] )
    old_entropy = entropy_of_target(dataf[target_attr_name])
    return old_entropy - new_entropy


print('Info-gain for buying is :'+str( info_gain(train_df, 'buying', 'decision')),"\n")
print('\n Info-gain for maint is: ' + str( info_gain(train_df, 'maint', 'decision')),"\n")
print('\n Info-gain for doors is:' + str( info_gain(train_df, 'doors', 'decision')),"\n")
print('\n Info-gain for persons is:' + str( info_gain(train_df, 'persons','decision')),"\n")
print('\n Info-gain for lug_boot is:' + str( info_gain(train_df, 'lug_boot','decision')),"\n")
print('\n Info-gain for safety is:' + str( info_gain(train_df, 'safety','decision')),"\n")




Information Gain Calculation of  buying

 No of Instances of the Sub Class is 343.0:

 Classes: 0 3
 
 Probabilities of Class 0 is 0.08746355685131195:
 
 Probabilities of Class 3 is 0.6268221574344023:

 No of Instances of the Sub Class is 349.0:

 Classes: 0 3
 
 Probabilities of Class 0 is 0.06303724928366762:
 
 Probabilities of Class 3 is 0.5931232091690545:

 No of Instances of the Sub Class is 344.0:

 Classes: 0 1
 
 Probabilities of Class 0 is 0.25290697674418605:
 
 Probabilities of Class 1 is 0.747093023255814:

 No of Instances of the Sub Class is 348.0:

 Classes: 0 1
 
 Probabilities of Class 0 is 0.16954022988505746:
 
 Probabilities of Class 1 is 0.8304597701149425:

 No of Instances of the Sub Class is 1384.0:

 Classes: 0 3
 
 Probabilities of Class 0 is 0.03757225433526012:
 
 Probabilities of Class 3 is 0.6994219653179191:
Info-gain for buying is :0.0963457468586768 

Information Gain Calculation of  maint

 No of Instances of the Sub Class is 349.0:

 Classes: 0 3


# ID3 Algorithm

In [132]:
def ID3(dataf, target_attr_name, attr_names, default_class=None):
    
    #calculating target attribute:
        from collections import Counter
        count = Counter(x for x in dataf[target_attr_name])# class of YES /NO
       
    ## First checking if the split of the dataset homogeneous
        if len(count) == 1:
             return next(iter(count))  # next input data set, or raises StopIteration when EOF is hit
    
    ## Second checking if the split of the dataset is empty
    # if yes, return a default value
        elif dataf.empty or (not attr_names):
            return default_class  # Return None for Empty Data Set
    
    
        else:
        #Default Value for next call
            default_class = max(count.keys()) #No of YES and NO Class
            
        # Computing the Information Gain of the attributes
            gainz = [info_gain(dataf, attr, target_attr_name) for attr in attr_names] 
            index_of_max = gainz.index(max(gainz)) # Index of Best Attribute
            
        # Choosing Best Attribut
            best_attr = attr_names[index_of_max]
        
        # Creating an empty tree, to be populated in a moment
            tree = {best_attr:{}} # Iniiate the tree with best attribute as a node 
            remaining_attr_names = [i for i in attr_names if i != best_attr]
        
        
        # populating the empty tree with subtrees
        
            for attr_val, data_subset in dataf.groupby(best_attr):
                subtree = ID3(data_subset,
                        target_attr_name,
                        remaining_attr_names,
                        default_class)
                tree[best_attr][attr_val] = subtree
        return tree

# Predicting Attributes

In [133]:

attr_names = list(train_df.columns)
print("List of Attributes:", attr_names) 
attr_names.remove('decision') #Remove the class attribute 
print("Predicting Attributes:", attr_names)

List of Attributes: ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'decision', 'kfold']
Predicting Attributes: ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'kfold']


In [134]:
# Run Algorithm:
from pprint import pprint
tree = ID3(test_df,'decision',attr_names)
print("\n\nThe Resultant Decision Tree is :\n")
pprint(tree)
attr = next(iter(tree))
print("Best Attribute :\n",attr)
print("Tree Keys:\n",tree[attr].keys())

Information Gain Calculation of  buying

 No of Instances of the Sub Class is 89.0:

 Classes: 0 3
 
 Probabilities of Class 0 is 0.10112359550561797:
 
 Probabilities of Class 3 is 0.48314606741573035:

 No of Instances of the Sub Class is 83.0:

 Classes: 0 3
 
 Probabilities of Class 0 is 0.012048192771084338:
 
 Probabilities of Class 3 is 0.7349397590361446:

 No of Instances of the Sub Class is 88.0:

 Classes: 0 1
 
 Probabilities of Class 0 is 0.23863636363636365:
 
 Probabilities of Class 1 is 0.7613636363636364:

 No of Instances of the Sub Class is 84.0:

 Classes: 0 1
 
 Probabilities of Class 0 is 0.15476190476190477:
 
 Probabilities of Class 1 is 0.8452380952380952:

 No of Instances of the Sub Class is 344.0:

 Classes: 0 3
 
 Probabilities of Class 0 is 0.0377906976744186:
 
 Probabilities of Class 3 is 0.7034883720930233:
Information Gain Calculation of  maint

 No of Instances of the Sub Class is 83.0:

 Classes: 0 3
 
 Probabilities of Class 0 is 0.06024096385542168

# Classification Accuracy

In [135]:
def classification_accuracy(instance, tree, default=None): # Instance of mushsroom data with Predicted 
    
    
    attr = next(iter(tree))        
    print("Key:",tree.keys())  
    print("Attribute:",attr)
   
   
    if instance[attr] in tree[attr].keys():
        result = tree[attr][instance[attr]]
        print("Instance Attribute:",instance[attr],"TreeKeys :",tree[attr].keys())
        if isinstance(result, dict): # this is a tree, delve deeper
            return classification_accuracy(instance, result)
        else:
            return result # this is a label
    else:
        return default

In [136]:
test_df['predicted'] = test_df.apply(classification_accuracy, axis=1, args=(tree,'No') ) 
    

print(test_df['predicted'])

print('\n Accuracy is:\n' + str( sum(test_df['decision'] == test_df['predicted'] ) / (1.0*len(test_df.index)) ))


test_df[['decision', 'predicted']]

Key: dict_keys(['safety'])
Attribute: safety
Instance Attribute: 1 TreeKeys : dict_keys([0, 1, 2])
Key: dict_keys(['persons'])
Attribute: persons
Instance Attribute: 4 TreeKeys : dict_keys([2, 4, 5])
Key: dict_keys(['maint'])
Attribute: maint
Instance Attribute: 1 TreeKeys : dict_keys([1, 2, 3, 4])
Key: dict_keys(['buying'])
Attribute: buying
Instance Attribute: 2 TreeKeys : dict_keys([1, 2, 3, 4])
Key: dict_keys(['doors'])
Attribute: doors
Instance Attribute: 3 TreeKeys : dict_keys([3, 4, 6])
Key: dict_keys(['safety'])
Attribute: safety
Instance Attribute: 2 TreeKeys : dict_keys([0, 1, 2])
Key: dict_keys(['persons'])
Attribute: persons
Instance Attribute: 5 TreeKeys : dict_keys([2, 4, 5])
Key: dict_keys(['buying'])
Attribute: buying
Instance Attribute: 2 TreeKeys : dict_keys([1, 2, 3, 4])
Key: dict_keys(['lug_boot'])
Attribute: lug_boot
Instance Attribute: 0 TreeKeys : dict_keys([0, 1, 2])
Key: dict_keys(['safety'])
Attribute: safety
Instance Attribute: 2 TreeKeys : dict_keys([0, 1, 2

C:\Users\Sharon\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,decision,predicted
1,1,1
5,1,1
6,0,0
14,0,0
17,1,1
...,...,...
1700,0,0
1701,0,0
1712,3,3
1713,3,3


In [139]:
Accuracy = sum(test_df['decision'] == test_df['predicted'] ) / (1.0*len(test_df.index))
#Std = np.std(test_df['predicted'])
print('\n Accuracy is:', Accuracy * 100)
print("Standard Deviation is: %.2f " % np.std(test_df['predicted']))

#print("Standard Deviation of ANN: %.5f " % np.std(row/correct_instances))



 Accuracy is: 100.0
Standard Deviation is: 0.74 
